<a href="https://colab.research.google.com/github/jda-21/AI4ENG/blob/main/07%20-%20modelo%20ensemble%20LightGBM%20y%20Catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lightgbm catboost tqdm

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

DATA_PATH = "train_clean_unicode.csv"
TEST_PATH = "test_clean_unicode.csv"

print("Cargando TRAIN:", DATA_PATH)
df = pd.read_csv(DATA_PATH)
print(df.shape)

TARGET_COL = "RENDIMIENTO_GLOBAL_NUM"
assert TARGET_COL in df.columns, "No se encuentra el target."

df = df.dropna(subset=[TARGET_COL]).copy()
df[TARGET_COL] = df[TARGET_COL].astype(int)
df = df[df[TARGET_COL].isin([1,2,3,4])]

print("\nConteo de clases:")
print(df[TARGET_COL].value_counts().sort_index())


num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if TARGET_COL not in num_cols:
    num_cols.append(TARGET_COL)

df_num = df[num_cols].copy()
X = df_num.drop(columns=[TARGET_COL])
y = df_num[TARGET_COL]

X = X.fillna(X.median())

X = X.astype(np.float32)

print("Shapes -> X:", X.shape, "| y:", y.shape)


X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape, "| Val:", X_val.shape)


lgbm = LGBMClassifier(
    objective="multiclass",
    num_class=4,
    boosting_type="gbdt",
    learning_rate=0.05,
    n_estimators=2000,
    max_depth=-1,
    num_leaves=63,
    min_data_in_leaf=50,
    reg_lambda=1.0,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

print("\nEntrenando LightGBM…")
lgbm.fit(X_train, y_train)


print("\nPredicción LightGBM…")
batch = 20000
lgbm_probs_list = []

for i in tqdm(range(0, len(X_val), batch)):
    part = X_val.iloc[i:i+batch]
    lgbm_probs_list.append(lgbm.predict_proba(part))

probs_lgbm = np.vstack(lgbm_probs_list)
pred_lgbm = probs_lgbm.argmax(axis=1) + 1

acc_lgbm = accuracy_score(y_val, pred_lgbm)
f1_lgbm = f1_score(y_val, pred_lgbm, average="macro")

print(f"LightGBM → acc={acc_lgbm:.4f} | f1_macro={f1_lgbm:.4f}")

# Entrenar CatBoost

print("\nEntrenando CatBoost…")

cat = CatBoostClassifier(
    loss_function="MultiClass",
    depth=6,
    learning_rate=0.05,
    l2_leaf_reg=5,
    iterations=640,
    random_seed=42,
    verbose=100
)

cat.fit(X_train, y_train)

print("\nPredicción CatBoost…")
cat_probs_list = []

for i in tqdm(range(0, len(X_val), batch)):
    part = X_val.iloc[i:i+batch]
    cat_probs_list.append(cat.predict_proba(part))

probs_cat = np.vstack(cat_probs_list)
pred_cat = probs_cat.argmax(axis=1) + 1

acc_cat = accuracy_score(y_val, pred_cat)
f1_cat = f1_score(y_val, pred_cat, average="macro")

print(f"CatBoost → acc={acc_cat:.4f} | f1_macro={f1_cat:.4f}")

# Ensemble
# =========================================================
probs_ens = (probs_lgbm + probs_cat) / 2
pred_ens = probs_ens.argmax(axis=1) + 1

acc_ens = accuracy_score(y_val, pred_ens)
f1_ens = f1_score(y_val, pred_ens, average="macro")

print(f"\nEnsemble → acc={acc_ens:.4f} | f1_macro={f1_ens:.4f}")


print("\nCargando TEST:", TEST_PATH)
test_df = pd.read_csv(TEST_PATH)

# usar solo columnas numéricas
test_df_num = test_df.select_dtypes(include=[np.number]).fillna(0).astype(np.float32)

# predicción con barra
print("\nPredicción FINAL (ensemble)…")
final_probs_list = []
for i in tqdm(range(0, len(test_df_num), batch)):
    part = test_df_num.iloc[i:i+batch]
    p_l = lgbm.predict_proba(part)
    p_c = cat.predict_proba(part)
    p = (p_l + p_c) / 2
    final_probs_list.append(p)

final_probs = np.vstack(final_probs_list)
final_pred = final_probs.argmax(axis=1) + 1

# submission
sub = pd.DataFrame({
    "ID": test_df["ID"],
    "RENDIMIENTO_GLOBAL": final_pred
})

sub.to_csv("submission_ensemble_colab.csv", index=False)

Cargando TRAIN: train_clean_unicode.csv
(692500, 47)

Conteo de clases:
RENDIMIENTO_GLOBAL_NUM
1    172987
2    172275
3    171619
4    175619
Name: count, dtype: int64
Shapes -> X: (692500, 46) | y: (692500,)
Train: (554000, 46) | Val: (138500, 46)

Entrenando LightGBM…
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050998 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1465
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 42
[LightGBM] [Info] Start training from score -1.387089
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Star

  0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50


 14%|█▍        | 1/7 [00:16<01:41, 16.84s/it]

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50


...

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50


 93%|█████████▎| 14/15 [03:56<00:16, 16.30s/it]

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50


100%|██████████| 15/15 [04:09<00:00, 16.64s/it]



✔ Archivo 'submission_ensemble_colab.csv' listo para descargar.
